许多您使用 LangChain 构建的应用程序将包含多个步骤和多次调用LLM。随着这些应用程序变得越来越复杂，能够检查您的链或代理内部究竟发生了什么变得至关重要。最佳方式是使用LangSmith。

在您点击上面的链接注册后，请确保设置环境变量以开始记录跟踪：

In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key="sk-f3c14e0485944adbbeb9b6fc26d930f7",
                   model="qwen-plus",
                   base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

首先直接使用模型。`ChatModel` 是 LangChain "可运行实例"，这意味着它们提供了一个标准接口来与之交互。要简单地调用模型，我们可以向 `.invoke` 方法传递一条消息列表。

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

it does not have a state concept itself. 例如，如果你问一个后续问题：

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

让我们看看这个例子 LangSmith 跟踪

我们可以看到它没有将之前的对话内容转化为上下文，也无法回答问题。这导致聊天机器人体验极差！

为了解决这个问题，我们需要将整个对话历史传递给模型。让我们看看这样做会发生什么：

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

现在我们可以看到我们得到了良好的响应！

这是支撑聊天机器人进行对话交互的基本理念。那么我们如何最好地实现这一点呢？

# 消息持久性
LangGraph 实现了一个内置的持久化层，使其非常适合支持多轮对话的聊天应用。

将我们的聊天模型封装在最小化的 LangGraph 应用程序中，使我们能够自动持久化消息历史，简化多轮应用的开发。

LangGraph 附带一个简单的内存检查点器，我们下面会用到。请参阅其文档获取更多详细信息，包括如何使用不同的持久化后端（例如 SQLite 或 Postgres）。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们现在需要创建一个配置，每次运行可执行文件时都要传递它。这个配置包含的信息不是直接输入的一部分，但仍然很有用。在这种情况下，我们希望包括一个线程 ID。它应该看起来像：

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

这使我们能够使用单个应用程序支持多个对话线程，这是当您的应用程序有多个用户时的常见需求。

我们可以调用该应用程序：

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

太棒了！我们的聊天机器人现在能记住关于我们的信息了。如果我们更改配置以引用不同的 thread_id，我们就可以看到它从头开始新的对话。

In [ ]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

然而，我们总是可以回到原始对话（因为我们将其持久化存储在数据库中）

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

这是我们可以支持聊天机器人与许多用户进行对话的方法！

为支持异步操作，将 call_model 节点更新为异步函数，并在调用应用程序时使用 .ainvoke：

In [ ]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

提示模板
提示模板有助于将原始用户信息转换为LLM可以处理的形式。在这种情况下，原始用户输入只是一个消息，我们将其传递给LLM。现在让我们使它变得更加复杂。首先，我们添加一个包含一些自定义指令的系统消息（但仍然以消息作为输入）。接下来，我们将添加更多输入，而不仅仅是消息。

为了添加系统消息，我们将创建一个 ChatPromptTemplate。我们将使用 MessagesPlaceholder 来传递所有消息。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

我们现在可以将我们的应用程序更新以包含此模板：

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们以相同的方式调用应用程序：

In [ ]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

太棒了！现在让我们使我们的提示变得更加复杂。假设提示模板现在看起来像这样：

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

请注意，我们在提示中添加了一个新的 语言 输入。我们的应用程序现在有两个参数——输入 消息 和 语言。我们应该更新应用程序的状态以反映这一点：

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

请注意，整个状态都会被持久化，因此如果不需要更改，我们可以省略像 language 这样的参数：

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

管理对话历史
在构建聊天机器人时，理解一个重要概念是如何管理对话历史。如果未进行管理，消息列表将无限增长，并可能超出LLM的上下文窗口。因此，添加一个限制传递消息大小的步骤非常重要。

重要地，您应该在提示模板之前、从消息历史中加载之前的消息之后进行此操作。

我们可以通过在提示符前添加一个简单的步骤来修改messages键，然后将这个新链包装在消息历史类中。

LangChain 自带了一些内置助手，用于管理消息列表。在这种情况下，我们将使用trim_messages助手来减少发送给模型的消息数量。修剪器允许我们指定要保留多少个标记，以及其他参数，例如是否始终保留系统消息以及是否允许部分消息：

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

要在我们的链中使用它，我们只需在我们将消息输入传递到我们的提示之前运行修剪器即可。

In [ ]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

现在如果我们尝试询问模型我们的名字，它不会知道，因为我们已经剪掉了聊天记录中的那部分：

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

但是如果我们询问关于最后几条消息中的信息，它会记住：

In [ ]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

 流媒体
现在我们有一个功能齐全的聊天机器人。然而，对于聊天机器人应用来说，有一个非常重要的用户体验考虑因素是流式传输。《LLMs》（大型语言模型）有时需要一些时间来响应，因此为了提高用户体验，大多数应用程序会做的一件事是随着每个标记的生成而流式传输它。这使用户能够看到进度。

实际上做这件事超级简单！

默认情况下，我们的 LangGraph 应用程序中的.stream流应用程序步骤——在这种情况下，模型响应的单个步骤。设置stream_mode="messages"允许我们流式传输输出标记：

In [ ]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")